# Demonstrations

In [ ]:
using IsingModel
using IsingModel.SpinSystems: calcEnergy
using IsingModel.SamplingHelper: makeSampler!
using BenchmarkTools
using Distributions
import Graphs
using GraphPlot: gplot
using LinearAlgebra
using Plots: histogram, plot, plot!
using Random
using Revise

In [ ]:
### When using NetworkX, uncomment the following lines.
# using Conda
# using PyCall
# using SparseArrays
# Conda.add("scipy")     # Make `Conda.add` run once the first time.
# Conda.add("networkx")
# const nx = pyimport("networkx")

## Ref: https://github.com/JuliaPy/PyCall.jl/issues/204
# const scipy_sparse_find = pyimport("scipy.sparse")["find"]
# function mysparse(Apy::PyObject)
#     IA, JA, SA = scipy_sparse_find(Apy)
#     return sparse(Int[i + 1 for i in IA], Int[i + 1 for i in JA], SA)
# end

In [ ]:
const N = 9  # The number of nodes
const SIDE_LENGTH = (Int ∘ ceil ∘ sqrt)(N)

### A way using the Graphs.jl library
G = Graphs.grid((SIDE_LENGTH, SIDE_LENGTH), periodic=true)
adjacencyMatrix = map(Graphs.adjacency_matrix(G)) do c
    ifelse(c == 0, 0, -1)
end

### Generate a square lattice with the periodic boundary condition by NetworkX.
# G = nx.grid_2d_graph(SIDE_LENGTH, SIDE_LENGTH, periodic=true)
# nx.set_edge_attributes(G, values=-1, name="weight")
# adjacencyMatrix = mysparse(nx.adjacency_matrix(G))

bias = zeros(N)
const INITIAL_CONFIGURATION = 2 .* rand(Bernoulli(0.5), N) .- 1
spinSystem = SpinSystems.SpinSystem(
    INITIAL_CONFIGURATION,
    adjacencyMatrix,
    bias
)
println("Hamiltonian = $(calcEnergy(spinSystem))")
pinningParameter = 0.5 * eigmax(collect(adjacencyMatrix))
spinSystemOnBipartiteGraph = SpinSystems.SpinSystemOnBipartiteGraph(
    INITIAL_CONFIGURATION,
    INITIAL_CONFIGURATION,
    0.5 * (adjacencyMatrix + pinningParameter * I),
    0.5 * bias,
    0.5 * bias
)
println("Modified Hamiltonian = $(calcEnergy(spinSystemOnBipartiteGraph) + 0.5 * pinningParameter * N)")

In [ ]:
gplot(G)

In [ ]:
const MAX_STEPS = N ^ 2 ÷ 2
const INITIAL_TEMPERATURE = float(N)
const FINAL_TEMPERATURE = 0.0

#annealingSchedule(n) = (FINAL_TEMPERATURE - INITIAL_TEMPERATURE) / MAX_STEPS * n + INITIAL_TEMPERATURE
annealingSchedule(n) = INITIAL_TEMPERATURE ^ (-n / MAX_STEPS)

function runAnnealer(
    Algorithm::Type{<:SpinSystems.UpdatingAlgorithm},
    spinSystem::SpinSystems.SpinSystem,
    INITIAL_TEMPERATURE::Float64=-1.0
)::Vector{Float64}
    if INITIAL_TEMPERATURE < 0.0
        return map(
            calcEnergy,
            makeSampler!(
                Algorithm(deepcopy(spinSystem)),
                MAX_STEPS,
                annealingSchedule=annealingSchedule
            )
        )
    else
        return map(
            calcEnergy,
            makeSampler!(
                Algorithm(deepcopy(spinSystem), INITIAL_TEMPERATURE),
                MAX_STEPS,
                annealingSchedule=annealingSchedule
            )
        )
    end
end

function runAnnealer(
    Algorithm::Type{<:SpinSystems.UpdatingAlgorithmOnBipartiteGraph},
    spinSystem::SpinSystems.SpinSystemOnBipartiteGraph,
    INITIAL_TEMPERATURE::Float64=-1.0
)::Vector{Float64}
    return map(
        a -> calcEnergy(a) + 0.5 * pinningParameter * N,
        makeSampler!(
            Algorithm(deepcopy(spinSystem), INITIAL_TEMPERATURE),
            MAX_STEPS,
            annealingSchedule=annealingSchedule
        )
    )
end

In [ ]:
@benchmark runAnnealer(SingleSpinFlip.GlauberDynamics, spinSystem, INITIAL_TEMPERATURE)

In [ ]:
plot(xlabel="MC steps", ylabel="Energy")
plot!(runAnnealer(SingleSpinFlip.AsynchronousHopfieldNetwork, spinSystem), label="Hopfield")
plot!(runAnnealer(SingleSpinFlip.GlauberDynamics, spinSystem, INITIAL_TEMPERATURE), label="Glauber")
plot!(runAnnealer(SingleSpinFlip.MetropolisMethod, spinSystem, INITIAL_TEMPERATURE), label="Metropolis")
plot!(runAnnealer(OnBipartiteGraph.StochasticCellularAutomata, spinSystemOnBipartiteGraph, INITIAL_TEMPERATURE), label="SCA")
plot!(runAnnealer(OnBipartiteGraph.MomentumAnnealing, spinSystemOnBipartiteGraph, INITIAL_TEMPERATURE), label="MA")

In [ ]:
histogram(
    map(
        (binary -> parse(Int, binary, base=2))
            ∘ join
            ∘ (spin -> (1 .- spin) .÷ 2) 
            ∘ SpinSystems.getSpinConfiguration,
        makeSampler!(SingleSpinFlip.GlauberDynamics(spinSystem, N / 4), 50000)
    ),
    normalize=:pdf
)
plot!(xlabel="Spin configuration", ylabel="Frequency")